In [1]:
import sys
import re
import numpy as np
import pandas as pd

## Load Data

In [2]:
# Input/output data files
input_file_cases = "processed_individual_cases_Sep20th2020_cleaned.csv.gz"
# input_file_locations = "processed_location_Sep20th2020.csv"
output_file_cases = "processed_individual_cases_Sep20th2020_cleaned.csv.gz"

In [3]:
# Load data from file(s)
individual_cases = pd.read_csv(input_file_cases, compression = 'gzip')
# locations = pd.read_csv(input_file_locations)

## Data Cleaning

In [4]:
# Select the non-NaN records
non_NaN_dates = individual_cases[['date_confirmation']].dropna()

### Convert Date

In [5]:
"""
Convert from range to a specific data.
The strategy here is to take the last day of the range.
"""
range_re = re.compile(r"^(\d\d).(\d\d).(\d\d\d\d)\s*-\s*(\d\d).(\d\d).(\d\d\d\d)$")
def range_to_date(s):
    m = range_re.match(s)
    if m:
        return m.group().split('-')[-1].strip()
    return s

non_NaN_dates['date_confirmation'] = non_NaN_dates['date_confirmation'].apply(range_to_date)

### Other formats

In [6]:
date_re = re.compile(r"^(\d\d).(\d\d).(\d\d\d\d)$")
def valid_date(s):
    if type(s) == str:
        return True if date_re.match(s) else False
    else:
        return False

non_NaN_dates[~non_NaN_dates['date_confirmation'].apply(valid_date)]

,date_confirmation


In [7]:
# Assign NaN to them
non_NaN_dates['date_confirmation'] = non_NaN_dates['date_confirmation'].apply(
    lambda x: x if valid_date(x) else np.NaN
)

In [8]:
# Add the cleaned data back to the original dataset
individual_cases['date_confirmation'] = non_NaN_dates

## Output Results

In [9]:
individual_cases.to_csv(output_file_cases, index = False, compression = 'gzip')